In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math

In [56]:
TARGET_POS = None


def on_mouse_click(event, x, y, flags, param):
    global TARGET_POS
    if event == cv2.EVENT_LBUTTONDOWN:
        TARGET_POS = (x, y)
        print(f"New Target Set: {TARGET_POS}")


def get_v_left_v_right(current_pos, heading_deg, target_pos):
    # 1. Setup constants (mimic your Unity variables)
    Kp = 1.0  # Proportional gain
    baseWidth = 2.5  # Distance between wheels (in pixels)
    wheelRadius = 1.5  # Wheel radius (in pixels)

    robotX, robotY = current_pos
    targetX, targetY = target_pos

    theta = math.radians(heading_deg)

    dx = (targetX - robotX) * Kp
    dy = -1 * (targetY - robotY) * Kp

    v_parallel = math.cos(theta) * dx + math.sin(theta) * dy
    v_perp = -math.sin(theta) * dx + math.cos(theta) * dy

    velocity = v_parallel
    omega = v_perp

    vRight = (velocity + (omega * baseWidth) / 2) / wheelRadius
    vLeft = (velocity - (omega * baseWidth) / 2) / wheelRadius

    vRight = max(-1, min(1, vRight))
    vLeft = max(-1, min(1, vLeft))

    return vLeft, vRight


def send_controls(vLeft, vRight):
    """
    Blueprint: Logic to send data to Unity or Hardware.
    """
    # print(f"Sending: L:{vLeft} R:{vRight}")
    pass

In [57]:
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
parameters = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(dictionary, parameters)

In [61]:
cap = cv2.VideoCapture("http://192.168.1.11:4747/video")
cv2.namedWindow("CodeRover")
cv2.setMouseCallback("CodeRover", on_mouse_click)

if not cap.isOpened():
    print('Capture not opened')

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    corners, ids, _ = detector.detectMarkers(frame)

    if ids is not None:
        for i, marker_corners in enumerate(corners):
            # 1. Get Center Position (Average of 4 corners)
            c = marker_corners[0]
            center_x = int(np.mean(c[:, 0]))
            center_y = int(np.mean(c[:, 1]))

            # [0]: top left RED, [1]: top right, [2]: bottom right, [3] bottom left
            front_pt = (c[0] + c[1]) / 2
            back_pt = (c[3] + c[2]) / 2
            dx = front_pt[0] - back_pt[0]
            dy = front_pt[1] - back_pt[1]
            
            lineMagPx = math.sqrt(dx**2 + dy**2) # in_pixels 50mm
            mm_per_pixel = 50 / lineMagPx

            angle = math.degrees(math.atan2(dy, dx))
            heading = (angle + 360) % 360

            # 3. Visuals
            cv2.circle(frame, (center_x, center_y), 5, (0, 255, 0), -1)
            cv2.line(frame, (int(back_pt[0]), int(back_pt[1])), (int(front_pt[0]), int(front_pt[1])), (255, 0, 0), 2)

            # 4. Control Loop
            if TARGET_POS:
                magToTargetPx = math.sqrt((TARGET_POS[0] - center_x)**2 + (TARGET_POS[1] - center_y)**2)
                magToTargetmm = magToTargetPx * mm_per_pixel
                
                if magToTargetmm >= 210:
                    cv2.line(frame, (center_x, center_y), TARGET_POS, (0, 255, 255), 1)
                    vL, vR = get_v_left_v_right((center_x, center_y), heading, TARGET_POS)
                    print(f"vRight: {vR}, vLeft: {vL}, magToTargetMeter: {magToTargetmm * 10**-3}")
                    send_controls(vL, vR)
                else:
                    print('reached!')
                    send_controls(0, 0)

    if TARGET_POS:
        cv2.circle(frame, TARGET_POS, 10, (0, 0, 255), 2)

    cv2.imshow("CodeRover", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

New Target Set: (39, 591)
vRight: 1, vLeft: -1, magToTargetMeter: 0.29798495897501864
vRight: 1, vLeft: -1, magToTargetMeter: 0.29605109356325643
vRight: 1, vLeft: -1, magToTargetMeter: 0.296534146431739
vRight: 1, vLeft: -1, magToTargetMeter: 0.296534146431739
vRight: 1, vLeft: -1, magToTargetMeter: 0.29352120334219767
vRight: 1, vLeft: -1, magToTargetMeter: 0.296534146431739
vRight: 1, vLeft: -1, magToTargetMeter: 0.29352120334219767
vRight: 1, vLeft: -1, magToTargetMeter: 0.2966263649805189
vRight: 1, vLeft: -1, magToTargetMeter: 0.2966263649805189
vRight: 1, vLeft: -1, magToTargetMeter: 0.29674741226773677
vRight: 1, vLeft: -1, magToTargetMeter: 0.29352120334219767
vRight: 1, vLeft: -1, magToTargetMeter: 0.2966263649805189
vRight: 1, vLeft: -1, magToTargetMeter: 0.29674741226773677
vRight: 1, vLeft: -1, magToTargetMeter: 0.29413624906244373
vRight: 1, vLeft: -1, magToTargetMeter: 0.2966263649805189
vRight: 1, vLeft: -1, magToTargetMeter: 0.29605109356325643
vRight: 1, vLeft: -1, ma

In [ ]:
cap.release()
cv2.destroyAllWindows()